In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score

In [4]:
from google.colab import files
uploaded = files.upload()

Saving word2vec100.pkl to word2vec100.pkl


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import zipfile
import os

# Assuming you uploaded 'my_folder.zip'
with zipfile.ZipFile('/content/drive/MyDrive/Copy of final_dataset_for_model_Training (1).zip', 'r') as zip_ref:
    zip_ref.extractall('my_folder')


In [5]:
df_copy = pd.read_csv('/content/my_folder/final_dataset_for_model_Training.csv')

In [6]:
X = df_copy.iloc[:, 1:]
y = df_copy.iloc[:, :1]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

In [16]:
X.shape

(404264, 220)

In [ ]:
xgbModel = XGBClassifier(tree_method = 'gpu_hist', predictor = 'gpu_predictor', gpu_id = 0, use_label_encoder = False, eval_metric = 'logloss', random_state = 42)

xgbParams = {
    'n_estimators': [100, 200],
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}

xgbGrid = RandomizedSearchCV(xgbModel, xgbParams, cv = 3, scoring='accuracy', n_jobs = -1, verbose = 1)
xgbGrid.fit(X_train, y_train)

print(classification_report(y_test, xgbGrid.predict(X_test)))

Fitting 3 folds for each of 10 candidates, totalling 30 fits


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [11:37:29] WARNING: /workspace/src/common/error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [11:37:29] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [11:37:29] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "predictor", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


AttributeError: 'RandomizedSearchCV' object has no attribute 'best_params'

In [9]:
print(xgbGrid.best_params_)

{'subsample': 1.0, 'n_estimators': 200, 'max_depth': 10, 'learning_rate': 0.1}


In [10]:
print(accuracy_score(y_test, xgbGrid.predict(X_test)))

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:2676: UserWarning: [11:52:36] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  if len(data.shape) != 1 and self.num_features() != data.shape[1]:
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:729: UserWarning: [11:52:36] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


0.8401539587992005


In [14]:
print(classification_report(y_test, xgbGrid.predict(X_test)))
print(xgbGrid.n_features_in_)

              precision    recall  f1-score   support

           0       0.87      0.87      0.87     63947
           1       0.78      0.78      0.78     37119

    accuracy                           0.84    101066
   macro avg       0.83      0.83      0.83    101066
weighted avg       0.84      0.84      0.84    101066

220


In [12]:
import pickle
with open('model.pkl', 'wb') as f:
  pickle.dump(xgbGrid, f)

In [13]:
from google.colab import files
files.download('model.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
xgbModel1 = XGBClassifier(tree_method = 'gpu_hist', predictor = 'gpu_predictor', gpu_id = 0, use_label_encoder = False, eval_metric = 'logloss', random_state = 42)

xgbParams = {
    'n_estimators': [100, 200],
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}

xgbGrid1 = GridSearchCV(xgbModel, xgbParams, cv = 3, scoring='accuracy', n_jobs = -1, verbose = 1)
xgbGrid.fit(X_train, y_train)



print(classification_report(y_test, xgbGrid1.predict(X_test)))

Fitting 3 folds for each of 10 candidates, totalling 30 fits


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [12:05:21] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [12:05:21] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "predictor", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:2676: UserWarning: [12:05:36] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  if len(data.shape) != 1 and self.num_features() != data.shape[1]:


              precision    recall  f1-score   support

           0       0.87      0.87      0.87     63947
           1       0.78      0.78      0.78     37119

    accuracy                           0.84    101066
   macro avg       0.83      0.83      0.83    101066
weighted avg       0.84      0.84      0.84    101066

